<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/999_Optimized_Trend_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener backtesting

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def Optimized_Trend_Tracker(data, prt, prc):
    df = data.copy()
    pds = prt
    percent = prc
    alpha = 2 / (pds + 1)

    df['ud1'] = np.where(df['Close'] > df['Close'].shift(1), (df['Close'] - df['Close'].shift()), 0)
    df['dd1'] = np.where(df['Close'] < df['Close'].shift(1), (df['Close'].shift() - df['Close']), 0)
    df['UD'] = df['ud1'].rolling(9).sum()
    df['DD'] = df['dd1'].rolling(9).sum()
    df['CMO'] = ((df['UD'] - df['DD']) / (df['UD'] + df['DD'])).fillna(0).abs()

    df['Var'] = 0.0
    for i in range(pds, len(df)):
        df['Var'].iat[i] = (alpha * df['CMO'].iat[i] * df['Close'].iat[i]) + (1 - alpha * df['CMO'].iat[i]) * df['Var'].iat[i - 1]

    df['fark'] = df['Var'] * percent * 0.01
    df['newlongstop'] = df['Var'] - df['fark']
    df['newshortstop'] = df['Var'] + df['fark']
    df['longstop'] = 0.0
    df['shortstop'] = 999999999999999999

    for i in df['UD']:
        def maxlongstop():
            df.loc[(df['newlongstop'] > df['longstop'].shift(1)), 'longstop'] = df['newlongstop']
            df.loc[(df['longstop'].shift(1) > df['newlongstop']), 'longstop'] = df['longstop'].shift(1)
            return df['longstop']

        def minshortstop():
            df.loc[(df['newshortstop'] < df['shortstop'].shift(1)), 'shortstop'] = df['newshortstop']
            df.loc[(df['shortstop'].shift(1) < df['newshortstop']), 'shortstop'] = df['shortstop'].shift(1)
            return df['shortstop']

        df['longstop'] = np.where(((df['Var'] > df['longstop'].shift(1))), maxlongstop(), df['newlongstop'])
        df['shortstop'] = np.where(((df['Var'] < df['shortstop'].shift(1))), minshortstop(), df['newshortstop'])

    df['xlongstop'] = np.where(((df['Var'].shift(1) > df['longstop'].shift(1)) & (df['Var'] < df['longstop'].shift(1))), 1, 0)
    df['xshortstop'] = np.where(((df['Var'].shift(1) < df['shortstop'].shift(1)) & (df['Var'] > df['shortstop'].shift(1))), 1, 0)

    df['trend'] = 0
    df['dir'] = 0

    for i in df['UD']:
        df['trend'] = np.where(((df['xshortstop'] == 1)), 1, (np.where((df['xlongstop'] == 1), -1, df['trend'].shift(1))))
        df['dir'] = np.where(((df['xshortstop'] == 1)), 1, (np.where((df['xlongstop'] == 1), -1, df['dir'].shift(1).fillna(1))))

    df['MT'] = np.where(df['dir'] == 1, df['longstop'], df['shortstop'])
    df['OTT'] = np.where(df['Var'] > df['MT'], (df['MT'] * (200 + percent) / 200), (df['MT'] * (200 - percent) / 200))

    df = df.round(2)
    df['OTT2'] = df['OTT'].shift(2)
    df['OTT3'] = df['OTT'].shift(3)
    df['Entry'] = df['Var'] > df['OTT2']
    df['Exit'] = df['Var'] < df['OTT2']
    return df


tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

#Raporlama için kullanılacak başlıklar
Titles = ['Hisse Adı', 'Son Fiyat','Giriş Sinyali', 'Çıkış Sinyali']

df_signals = pd.DataFrame(columns=Titles)

for i in range(0,len(Hisseler)):
    #print(Hisseler[i])
    try:
        data = tv.get_hist(symbol=Hisseler[i], exchange='BIST', interval=Interval.in_1_hour, n_bars=100)
        data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
        data = data.reset_index()
        OTT = Optimized_Trend_Tracker(data,2,1.4)
        OTT['datetime'] = pd.to_datetime(OTT['datetime'])  # Assuming 'Date' is the name of your datetime column
        OTT.set_index('datetime', inplace=True)
        Buy = False
        Sell = False
        Signals = OTT.tail(2)
        Signals = Signals.reset_index()
        Buy = Signals.loc[0, 'Entry'] == False and Signals.loc[1, 'Entry'] == True
        Sell = Signals.loc[0, 'Exit'] == False and Signals.loc[1, 'Exit'] == True
        Last_Price = Signals.loc[1, 'Close']
        L1 = [Hisseler[i] ,Last_Price, str(Buy), str(Sell)]
        df_signals.loc[len(df_signals)] = L1
        print(L1)
    except:
        pass

df_True = df_signals[(df_signals['Giriş Sinyali'] == 'True')]
print(df_True)

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-z3e_y1lr
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-z3e_y1lr
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 53.1 MB/s eta 0:00:00
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=3b62627594de745bedbf17a7cfc086338dadf89afa3601bf72562a780fefc093
  Stored in directory: /tmp/pip-ephem-wheel-cache-xl5zjl1t/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


['A1CAP', 26.52, 'False', 'False']
['ACSEL', 132.1, 'False', 'False']
['ADEL', 515.5, 'False', 'False']
['ADESE', 2.25, 'False', 'False']
['ADGYO', 27.5, 'False', 'False']
['AEFES', 205.1, 'False', 'False']
['AFYON', 12.7, 'False', 'False']
['AGESA', 112.2, 'False', 'False']
['AGHOL', 339.0, 'False', 'False']
['AGROT', 15.92, 'False', 'False']
['AGYO', 8.17, 'False', 'False']
['AHGAZ', 13.49, 'False', 'False']
['AKBNK', 59.3, 'False', 'False']
['AKCNS', 129.7, 'False', 'False']
['AKENR', 15.91, 'False', 'False']
['AKFGY', 2.33, 'False', 'False']
['AKFYE', 26.38, 'False', 'False']
['AKGRT', 8.18, 'False', 'False']
['AKMGY', 297.25, 'False', 'False']
['AKSA', 121.5, 'False', 'False']
['AKSEN', 46.42, 'True', 'False']
['AKSGY', 17.0, 'False', 'False']
['AKSUE', 12.81, 'False', 'False']
['AKYHO', 7.6, 'False', 'False']
['ALARK', 116.2, 'False', 'False']
['ALBRK', 5.42, 'False', 'False']
['ALBTS', 2662.0, 'False', 'False']
['ALCAR', 1285.0, 'False', 'False']
['ALCTL', 104.0, 'False', 'False